<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# The `interpolate_for_A_gauge_rhs` function

## Authors: Samuel Cupp, Leo Werneck, and Zach Etienne

<font color='red'>**This module is currently under development**</font>

### Required and recommended citations:
* **(Required)** GRHayL (TBD)
* **(Required)** [Etienne, Paschalidis, Haas, Mösta, and Shapiro. IllinoisGRMHD: an open-source, user-friendly GRMHD code for dynamical spacetimes (2015)](http://arxiv.org/abs/1501.07276).

<a id='introduction'></a>

# Introduction
$$\label{introduction}$$

The `ghl_interpolate_for_induction_rhs` function takes stencils of spacetime and electromagnetic quantities and interpolates them to the staggered positions of $A_i$ and $\tilde{\Phi}$. It then returns the interpolated quantities in the `induction_interp_vars` struct.

<a id='arglist'></a>

## Function Argument List
$$\label{arglist}$$

The prototype for the `ghl_interpolate_for_A_gauge_rhs` function is
```
void ghl_interpolate_for_induction_rhs(
      const metric_quantities metric_stencil[2][2][2],
      const double psi_stencil[2][2][2],
      const double Ax_stencil[3][3][3],
      const double Ay_stencil[3][3][3],
      const double Az_stencil[3][3][3],
      const double phitilde,
      induction_interp_vars *restrict interp_vars);
```

The first argument (`metric_stencil`) is an array of `metric_quantities` structs. The function requires the lapse, shift, and BSSN metric inverse.

The second argument (`psi_stencil`) is an array containing a stencil of the BSSN quantity $\psi$.

The next three arguments (`Ax_stencil`, `Ay_stencil`, `Az_stencil`) are arrays containing stencils for each component of $A_i$. These stencils range from $i-1$ to $i+1$ for each index. Technically, the function only uses the stencils
```
A_x: { 0,1, -1,1, -1,1}
A_y: {-1,1,  0,1, -1,1}
A_z: {-1,1, -1,1,  0,1}
```
but making the arrays cubes allows for easier filling of the stencil.

The final input argument (`phitilde`) is the value of $\tilde{\Phi}$ at the current gridpoint.

The output is stored in the struct `interp_vars`, which is defined as
```
typedef struct induction_interp_vars {
  double alpha_interp;
  double alpha_Phi_minus_betaj_A_j_interp;
  double sqrtg_Ai_interp[3];
  double shifti_interp[3];
} induction_interp_vars;
```
The array `interp_vars->sqrtg_Ai_interp[3]` are interpolated to the staggered gridpoint of the corresponding component of $A_i$ for use in finite differences to compute $\tilde{\Phi}^\mathrm{RHS}$. The quantities `interp_vars->alpha_interp`, `interp_vars->shifti_interp[3]`, and `interp_vars->alpha_Phi_minus_betaj_A_j_interp` are interpolated to the fully staggered gridpoint of $\tilde{\Phi}$. The first two are used to compute $\tilde{\Phi}^\mathrm{RHS}$, while the latter is used in a finite difference scheme for computing $A_i^\mathrm{RHS}$.

<a id='inputs'></a>

## Providing Inputs
$$\label{inputs}$$

The first two arguments can be filled with a loop of the form
```
for(int iterz=0; iterz<2; iterz++) {
  for(int itery=0; itery<2; itery++) {
    for(int iterx=0; iterx<2; iterx++) {
      const int ind = grid_index(i+iterx,j+itery,k+iterz);
      metric_stencil[iterz][itery][iterx].lapse          = lapse[ind];
      metric_stencil[iterz][itery][iterx].betaU[0]       = betax[ind];
      metric_stencil[iterz][itery][iterx].betaU[1]       = betay[ind];
      metric_stencil[iterz][itery][iterx].betaU[2]       = betaz[ind];
      metric_stencil[iterz][itery][iterx].gammaUU[0][0]  = gupxx[ind];
      metric_stencil[iterz][itery][iterx].gammaUU[0][1]  = gupxy[ind];
      metric_stencil[iterz][itery][iterx].gammaUU[0][2]  = gupxz[ind];
      metric_stencil[iterz][itery][iterx].gammaUU[1][1]  = gupyy[ind];
      metric_stencil[iterz][itery][iterx].gammaUU[1][2]  = gupyz[ind];
      metric_stencil[iterz][itery][iterx].gammaUU[2][2]  = gupzz[ind];
      psi_stencil[iterz][itery][iterx] = psi[ind];
    }
  }
}
```
where `ind` represents the indexing for the grid function in the given infrastructure. Similarly, the $A_i$ stencils can be filled with
```
for(int iterz=-1; iterz<2; iterz++)
  for(int itery=-1; itery<2; itery++)
    for(int iterx=-1; iterx<2; iterx++) {
      const int ind = grid_index(i+iterx,j+itery,k+iterz);
      Ax_stencil[iterz+1][itery+1][iterx+1] = Ax[ind];
      Ay_stencil[iterz+1][itery+1][iterx+1] = Ay[ind];
      Az_stencil[iterz+1][itery+1][iterx+1] = Az[ind];
    }
  }
}
```
As previously mentioned, the full $A_i$ stencil is not actually used, so one can only fill the needed data.

<a id='inputs'></a>

# Function Internals
$$\label{inputs}$$

The function returns several interpolated quantities which are computed using averaged-based interpolations of the input data. For the RHS of $\tilde{\Phi}$, this means calculating
$$
\alpha \sqrt{\gamma} A^i = \alpha psi^6 A^i
$$
Since this code inherits from `IllinoisGRMHD` (which itself inherits from the original closed-source GRMHD code from the Illinois group), this is computed by interpolating the metric and $A_i$ to the location of each component of the vector potential (e.g. interpolating the metric, $A_y$, and $A_z$ to $(i, j+1/2, k+1/2)$ for the $A_x$ term). The returned quantities in `sqrtg_Ai_interp` are then computed by raising $A_i$ and multiplying by $\alpha \psi^2$.

Similarly, the RHS for $A_i$ needs the quantity
$$
\alpha \Phi - \beta^j A_j = \frac{\alpha}{\psi^6}\tilde{\Phi} - \beta^j A_j
$$
at the point $(i+\frac{1}{2},j+\frac{1}{2},k+\frac{1}{2})$. The above equation simply uses the returned quantities `alpha_interp` and `shifti_interp` and interpolated quantities for $A_i$.